# Registration matrix loading

Notebook to load registration matrixes ut data into the databse

## Imports

In [ ]:
import psycopg2
import numpy as np
#import a folder in the parent directory
import sys
sys.path.append('../')
import dbtools.dbtools as qrs
from pathlib import Path
import tifffile as tiff

## Connection

In [ ]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connected to the database


## Load measuements data

We need this table to get the id of the registered measurement, so we load it now to not load it twice

In [ ]:
measurements = qrs.get_data('measurements')

measurements.head(5)

,id_measurement,file_path_measurement,parent_measurement_id_measurement,measurementtype_id_measurement
0,1,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,NaN,3
1,3,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,1.0,3
2,4,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,1.0,3
3,5,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,1.0,3
4,6,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,1.0,3


## Select UT measurement

We have to get the id of the UT measurement to link the registration matrix to it

In [9]:
ut_file = Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Probetas JI\probetas\4\4.tif')

ut_id = measurements[measurements['file_path_measurement'] == str(ut_file)]['id_measurement'].values[0]

ut_id = int(ut_id)

ut_id 

3

## Select XCT measurement

We have to get the id of the XCT measurement to link the registration matrix to it

In [16]:
xct_file = Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Juan Ignacio\probetas\4\frontal_90right.tif')

xct_id = measurements[measurements['file_path_measurement'] == str(xct_file)]['id_measurement'].values[0]

xct_id = int(xct_id)

xct_id 

41

## Registration data

Now we have to set the attributes of the registration:

1. Main attributes

    1. reference_measurement_id: Is the measurement used as reference in the registration

    2. registered_measurement_id: The measurement registered to match reference

    3. registration_matrix: The result of the registration
    

In [17]:
main_parameters = {'reference_measurement_id':ut_id,'registered_measurement_id':xct_id}

registration_matrix = r'[[ 9.99924865e-01 -1.22582733e-02 -7.75030259e+02][ 1.22582733e-02  9.99924865e-01 -1.78468051e+02][ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]'

main_parameters['registration_matrix'] = registration_matrix


## Load into the table

In [18]:
print('Parameters to be inserted: ')
for key, value in main_parameters.items():
    print(f"-    {key}: {value}")

Parameters to be inserted: 
-    reference_measurement_id: 3
-    registered_measurement_id: 41
-    registration_matrix: [[ 9.99924865e-01 -1.22582733e-02 -7.75030259e+02][ 1.22582733e-02  9.99924865e-01 -1.78468051e+02][ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [19]:
table_name = 'measurement_registrations'

# Extract column names and values from the attributes dictionary
columns = ', '.join(main_parameters.keys())
values = ', '.join([f"'{v}'" for v in main_parameters.values()])

# Construct the SQL INSERT statement
sql = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"

print(sql)

# Create a cursor object using the cursor() method
cursor = conn.cursor()

# Execute the SQL statement
cursor.execute(sql)

cursor.execute('COMMIT')

cursor.close()

INSERT INTO measurement_registrations (reference_measurement_id, registered_measurement_id, registration_matrix) VALUES ('3', '41', '[[ 9.99924865e-01 -1.22582733e-02 -7.75030259e+02][ 1.22582733e-02  9.99924865e-01 -1.78468051e+02][ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]')


## Make sure insert was correct

In [ ]:
data = qrs.get_data(table_name)

data

,id_measurement_registration,reference_measurement_id_measurement_registration,registered_measurement_id_measurement_registration,registration_matrix_measurement_registration
0,1,3,41,[[ 9.99924865e-01 -1.22582733e-02 -7.75030259e...
